In [ ]:
# # 필요 패키지 설치
# !conda install -c plotly plotly --yes
# !pip install dash
# !pip install jupyter-dash

***dash에 맞게 데이터 처리***

In [ ]:
# import
import dash
from dash import dcc, Dash, html, Input, Output
import plotly.express as px
import pandas as pd
import plotly.offline as pyo # 그래프 보이지 않을 때 오프라인 모드로 변경하기 위함
pyo.init_notebook_mode() # 해당 코드 같이 실행 

# 데이터 가져오기 
def bring_test_2023_data():
    
    df = pd.read_csv("/Users/wonbinchoi/semiPJT_5/wonbin/oliveyoung_skincare.csv")
    df = df[df['brand_name'] == '닥터지'].copy() # 닥터지 데이터만 가져와서 테스트
    df['review_date'] = pd.to_datetime(df['dispRegDate']) # review_date로 datetime화
    
    # year,month 구분 컬럼 생성
    df['review_year'] = df['review_date'].dt.year
    df['review_month'] = df['review_date'].dt.month
    # 23년도 데이터로 테스트
    df = df[df['review_year'].isin([2023])].copy() 
    
    return df

# 필요한 Dropdown 리스트, Checklist 리스트 가져오기
def bring_needed_list(df):
    # Dropdown에 쓰일 year_list
    year_list = df['review_date'].dt.year.unique().tolist()
    year_list = sorted(year_list)
    # Dropdown에 쓰일 month_list
    month_list = df['review_date'].dt.month.unique().tolist()
    month_list = sorted(month_list)
    # Checklist에 쓰일 goodsNo_list ==> 실제에서는 brand_list가 될 예정 
    goodsNo_list = doctorg['goodsNo'].unique().tolist()
    
    return year_list, month_list, goodsNo_list

# test형 월별로 나눈 데이터 프레임 생성
def make_new_test_df(df):
    
    new_df = df.groupby(['goodsNo', 'review_month'])['gdasCont'].count().unstack()
    new_df.columns = ['Jan', 'Feb', 'Mar', 'Apr']
    new_df = new_df[:10].copy()
    return new_df


doctorg_23 = bring_test_2023_data()
year_list, month_list, goodsNo_list = bring_needed_list(doctorg_23)
new_df = make_new_test_df(doctorg_23)
new_df.to_csv('test_app_csv.csv', index=True)

***Dash 테스트***

In [ ]:
# test_app.py 에 dash 있음. 
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd

df = pd.read_csv('test_app_csv.csv')
goodsNo_list = df['goodsNo'].tolist()
app = Dash(__name__)
month_list = ['Jan', 'Feb', 'Mar', 'Apr']

app.layout = html.Div(children=[
    html.H1(children='고객리뷰 텍스트에 기반한 날짜별 올리브영 스킨케어 브랜드 평판 점수'),
    html.Div([
        '상품 번호',
        dcc.Checklist(id='goodsNo_list',
                     options=goodsNo_list,
                     value=goodsNo_list[:5]),
                     ]),
    html.Div([
        '날짜',
        dcc.Dropdown(id='review_month',
                     options=month_list,
                     value=month_list[0]),
                     ]),
    html.Br(),
    html.Div([dcc.Graph(id='update_graph')])
])

# callback
@app.callback(
    Output(component_id='update_graph', component_property='figure'),
    [Input(component_id='goodsNo_list', component_property='value'),
     Input(component_id='review_month', component_property='value')]
)
def update_graph(goodsNo, review_month):
    dff = df[df['goodsNo'].isin(goodsNo)] # 여기서 구분이 이루어져야 함. 
    fig = px.bar(dff, x='goodsNo', y=review_month, width=1000, height=700)
    return fig

if __name__ == '__main__':
    app.run_server(debug=False)

In [ ]:
!python test_app.py